In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from datetime import datetime

In [3]:
from ecephys.xrsig import get_emg
from sglxarray import load_trigger
import ecephys_analyses as ea

In [4]:
EMG_OPTS = dict(target_sf=20, window_size=25.0, wp=[300, 600], ws=[275, 625], gpass=1, gstop=60, ftype='butter')

In [ ]:
def get_condition_emg(subject, experiment, probe):
    emg_chans = ea.get_channels(subject, experiment, probe, "emg_from_lfp")
    
    bin_paths = ea.get_lfp_bin_paths(subject, experiment, probe=probe)
    emg_paths = ea.get_analysis_counterparts(bin_paths, "emg.nc", subject)
    #bin_paths = ea.get_sglx_style_datapaths(subject=subject, experiment=experiment, condition=condition, ext="lf.bin")
    #emg_paths = ea.get_sglx_style_datapaths(subject=subject, experiment=experiment, condition=condition, ext="emg.nc")

    for bin_path, emg_path in zip(bin_paths, emg_paths):
        sig = load_trigger(bin_path, emg_chans)
        emg = get_emg(sig, **EMG_OPTS)
        
        # Create parent directories if they do not already exist.
        Path(emg_path).parent.mkdir(parents=True, exist_ok=True)  
        emg.to_netcdf(emg_path)
        emg.close()
        
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"{current_time}: Finished {str(bin_path)}")

In [6]:
get_condition_emg("Adrian", "conveyor_over_water", "imec1")

nChan: 385, nFileSamp: 42859
Filtering LFP with wp=[300, 600], ws=[275, 625], gpass=1, gstop=60,filter type=butter
Computing EMG from filtered LFP...
target sf = 20, window size = 25.0, LFP sf=2500.0113678480425, LFP nchans = 3




Done!
17:47:24: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX11-Adrian/8-27-2021/SpikeGLX/8-27-2021_g0/8-27-2021_g0_imec1/8-27-2021_g0_t0.imec1.lf.bin
nChan: 385, nFileSamp: 175788
Filtering LFP with wp=[300, 600], ws=[275, 625], gpass=1, gstop=60,filter type=butter
Computing EMG from filtered LFP...
target sf = 20, window size = 25.0, LFP sf=2500.0113678480425, LFP nchans = 3




Done!
17:47:29: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX11-Adrian/8-27-2021/SpikeGLX/8-27-2021_g1/8-27-2021_g1_imec1/8-27-2021_g1_t0.imec1.lf.bin
nChan: 385, nFileSamp: 18000083
Filtering LFP with wp=[300, 600], ws=[275, 625], gpass=1, gstop=60,filter type=butter
Computing EMG from filtered LFP...
target sf = 20, window size = 25.0, LFP sf=2500.0113678480425, LFP nchans = 3




Done!
17:56:06: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX11-Adrian/8-27-2021/SpikeGLX/8-27-2021_g2/8-27-2021_g2_imec1/8-27-2021_g2_t0.imec1.lf.bin
nChan: 385, nFileSamp: 18000083
Filtering LFP with wp=[300, 600], ws=[275, 625], gpass=1, gstop=60,filter type=butter
Computing EMG from filtered LFP...
target sf = 20, window size = 25.0, LFP sf=2500.0113678480425, LFP nchans = 3




Done!
18:05:14: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX11-Adrian/8-27-2021/SpikeGLX/8-27-2021_g2/8-27-2021_g2_imec1/8-27-2021_g2_t1.imec1.lf.bin
nChan: 385, nFileSamp: 18000083
Filtering LFP with wp=[300, 600], ws=[275, 625], gpass=1, gstop=60,filter type=butter
Computing EMG from filtered LFP...
target sf = 20, window size = 25.0, LFP sf=2500.0113678480425, LFP nchans = 3


...



Done!
20:41:38: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX11-Adrian/8-28-2021/SpikeGLX/8-28-2021_g0/8-28-2021_g0_imec1/8-28-2021_g0_t8.imec1.lf.bin
nChan: 385, nFileSamp: 18000083
Filtering LFP with wp=[300, 600], ws=[275, 625], gpass=1, gstop=60,filter type=butter
Computing EMG from filtered LFP...
target sf = 20, window size = 25.0, LFP sf=2500.0113678480425, LFP nchans = 3




Done!
20:49:51: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX11-Adrian/8-28-2021/SpikeGLX/8-28-2021_g0/8-28-2021_g0_imec1/8-28-2021_g0_t9.imec1.lf.bin
nChan: 385, nFileSamp: 18000083
Filtering LFP with wp=[300, 600], ws=[275, 625], gpass=1, gstop=60,filter type=butter
Computing EMG from filtered LFP...
target sf = 20, window size = 25.0, LFP sf=2500.0113678480425, LFP nchans = 3




Done!
20:58:21: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX11-Adrian/8-28-2021/SpikeGLX/8-28-2021_g0/8-28-2021_g0_imec1/8-28-2021_g0_t10.imec1.lf.bin
nChan: 385, nFileSamp: 1824811
Filtering LFP with wp=[300, 600], ws=[275, 625], gpass=1, gstop=60,filter type=butter
Computing EMG from filtered LFP...
target sf = 20, window size = 25.0, LFP sf=2500.0113678480425, LFP nchans = 3




Done!
20:59:13: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX11-Adrian/8-28-2021/SpikeGLX/8-28-2021_g0/8-28-2021_g0_imec1/8-28-2021_g0_t11.imec1.lf.bin
nChan: 385, nFileSamp: 11257696
Filtering LFP with wp=[300, 600], ws=[275, 625], gpass=1, gstop=60,filter type=butter
Computing EMG from filtered LFP...
target sf = 20, window size = 25.0, LFP sf=2500.0113678480425, LFP nchans = 3




Done!
21:04:23: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX11-Adrian/8-28-2021/SpikeGLX/8-28-2021_g1/8-28-2021_g1_imec1/8-28-2021_g1_t0.imec1.lf.bin
